<a href="https://www.kaggle.com/code/fizcogar/intro-ia-5-nlp-2?scriptVersionId=115884112" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introducción a la Inteligencia Artificial
## Aprendizaje Profundo
### Procesamiento de Lenguaje Natural (NLP)

# Afinado (Fine tuning) de un modelo preentrenado para Clasificación en NLP

Un problema de clasificación de documentos es **aprendizaje máquina supervisado**: Partimos de un conjunto de datos que ya están clasificados 'a mano', por lo que **la clase de cada documento** forma parte de los datos de entrenamiento.

Además, los modelos que mejor se adaptan a estos problemas son arquitecturas de **redes neuronales** por lo que se clasifican dentro del **aprendizaje profundo**. De esta manera podemos partir de modelos preentrenados y aprovechar la **transferencia de aprendizaje**.


# Dataset de entrenamiento

El 'dataset' de entrenamiento es un conjunto de diagnósticos médicos ya clasificados según estén relacionados o no con la **odontología**.

Es un 'dataset' publicado en **huggingface**: https://huggingface.co/datasets/fvillena/spanish_diagnostics

## Entrenamiento con un **subconjunto** del 'dataset' de entrenamiento
En este caso vamos a seleccionar **una parte** del conjunto de datos de entrenamiento para poder experimentar con más agilidad: cada 'epoca' de entrenamiento va a ser mucho más rápida de ejecutar. Si el resultado es prometedor entrenaremos el modelo con todos los datos.

In [1]:
from datasets import load_dataset, concatenate_datasets, Features, Value

# Dataset de entrenamiento:
ds_original = load_dataset("fvillena/spanish_diagnostics")
ds_original = concatenate_datasets([ds_original['train'], ds_original['test']])
ds_original = ds_original.rename_columns({'label':'labels', 'text':'input'})
# Seleccionamos sólo una parte:
ds = ds_original.select(range(1000))
ds

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset spanish_diagnostics downloaded and prepared to /root/.cache/huggingface/datasets/fvillena___spanish_diagnostics/default/0.0.0/45c176cea64580ea9631f78c2867a657ede368597681e5337e9f1c976e4e84ff. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['input', 'labels'],
    num_rows: 1000
})

## Un par de ejemplos del conjunto de datos

Un ejemplo clasificado como '0', es decir, no es relativo a la odontología:

In [2]:
print("x: {}, y:{}".format(ds['input'][1], ds['labels'][1]))

x: OBTRUCCION FOSA NASAL DERECHA, y:0


Y un ejemplo clasificado como '1', es decir, relativo a la odontología:

In [3]:
print("x: {}, y:{}".format(ds['input'][9], ds['labels'][9]))

x: ASA 1 DENTICION TEMPORAL MORDIDA CRUZADA, y:1


# Entrenamiento:

Partimos de un modelo preentrenado en castellano. ¿Cuál? El que mejor resultado nos de. ¡Más experimentación!

Después de unas pruebas el modelo elegido es [Josue/BETO-espanhol-Squad2](https://huggingface.co/models?p=132&sort=downloads), otro modelo disponible en [Huggingface ](https://huggingface.co/models)

In [4]:
# MODELO PREENTRENADO
# En caso de no obtener buen resultado deberíamos probar con otros modelos en castellano
pre = 'Josue/BETO-espanhol-Squad2'

from transformers import AutoModelForSequenceClassification, AutoTokenizer, logging, TrainingArguments, Trainer, AutoConfig
logging.set_verbosity_error()
# Tokenizer y modelo:
tokz = AutoTokenizer.from_pretrained(pre)
model = AutoModelForSequenceClassification.from_pretrained(pre, num_labels=2)


Downloading:   0%|          | 0.00/135 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/465 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/237k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

## Preprocesado de los datos de entrada

Una red neuronal sólo sabe manejar **números**, pero aquí queremos procesar **texto**. Concretamente, la entrada de la red neuronal son **diagnósticos médicos** en lenguaje natural. ¿Cómo codificamos las entradas?

1. **Troceando** el texto en **'Tokens'**, que son, más o menos 'palabras'.
2. **Definiendo un vocabulario**, que es la lista numerada de todos los **'tokens'** que aparecen en los textos de entrenamiento. Cada 'token' tiene ahora un número asignado.
3. **Transformando** los textos que va a entrar a la red neuronal a la **lista de números** correspondiente según el vocabulario.

Para ello la librería, además del modelo, proporciona un 'tokenizador':

In [5]:
# Transformación de los textos:
def tok_func(x): return tokz(x["input"])
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

Aquí podemos ver 
1. Un diagnostico en texto natural
1. La lista de 'tokens' (que no son exactamente palabras)
1. La correspondiente lista de números que finalmente será le entrada a la red neuronal

In [6]:
print ("Entrada original: {}\n'Troceada': {}\nNumerizada: {}".format(ds['input'][9], tokz.tokenize(ds['input'][9]), tok_ds['input_ids'][9]))


Entrada original: ASA 1 DENTICION TEMPORAL MORDIDA CRUZADA
'Troceada': ['as', '##a', '1', 'den', '##tic', '##ion', 'temporal', 'mor', '##dida', 'cruzada']
Numerizada: [4, 1260, 30932, 1094, 1728, 3412, 2243, 7199, 2377, 2463, 26229, 5]


## Separación entre datos de entrenamiento y de validación:

Como siempre, separamos un porcentaje de datos para la validación:

In [7]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['input', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 750
    })
    test: Dataset({
        features: ['input', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 250
    })
})

## Métrica para la evaluación:
La 'exactitud' (accuracy) mide el porcentaje de aciertos del modelo. Es la otra cara de la moneda del ratio de error.

In [8]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# Entrenamiento

Entrenamos durante un par de épocas ...

In [9]:
# Número de épocas de entrenamiento
epochs = 2
# Tamaño del lote
bs = 20
# Ratio de aprendizaje
lr = 8e-5

args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=compute_metrics)

trainer.train()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input. If input are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 750
  Num Epochs = 2
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 1
  Total optimization steps = 76
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: in

{'eval_loss': 0.3010362982749939, 'eval_accuracy': 0.888, 'eval_runtime': 1.3305, 'eval_samples_per_second': 187.896, 'eval_steps_per_second': 5.261, 'epoch': 1.0}


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input. If input are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 250
  Batch size = 40


Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.3955020308494568, 'eval_accuracy': 0.876, 'eval_runtime': 1.3333, 'eval_samples_per_second': 187.507, 'eval_steps_per_second': 5.25, 'epoch': 2.0}
{'train_runtime': 15.3081, 'train_samples_per_second': 97.987, 'train_steps_per_second': 4.965, 'train_loss': 0.4151259221528706, 'epoch': 2.0}


TrainOutput(global_step=76, training_loss=0.4151259221528706, metrics={'train_runtime': 15.3081, 'train_samples_per_second': 97.987, 'train_steps_per_second': 4.965, 'train_loss': 0.4151259221528706, 'epoch': 2.0})

El resultado ('eval_accuracy') es **prometedor**, así que podemos entrenar con más datos (o incluso con todos)

In [10]:
# Troceado:
ds = ds_original.select(range(10000))
tok_ds = ds.map(tok_func, batched=True)
# Separación para validación:
dds = tok_ds.train_test_split(0.25, seed=42)
# Entrenamiento:
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=compute_metrics)
trainer.train()

  0%|          | 0/10 [00:00<?, ?ba/s]

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input. If input are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7500
  Num Epochs = 2
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 1
  Total optimization steps = 750
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: 

{'eval_loss': 0.30167171359062195, 'eval_accuracy': 0.9104, 'eval_runtime': 9.422, 'eval_samples_per_second': 265.336, 'eval_steps_per_second': 6.686, 'epoch': 1.0}


Saving model checkpoint to outputs/checkpoint-500
Configuration saved in outputs/checkpoint-500/config.json


{'loss': 0.3341, 'learning_rate': 2.501573626336352e-05, 'epoch': 1.33}


Model weights saved in outputs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-500/tokenizer_config.json
Special tokens file saved in outputs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input. If input are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2500
  Batch size = 40


Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.2595323324203491, 'eval_accuracy': 0.9256, 'eval_runtime': 9.4028, 'eval_samples_per_second': 265.877, 'eval_steps_per_second': 6.7, 'epoch': 2.0}
{'train_runtime': 148.4379, 'train_samples_per_second': 101.052, 'train_steps_per_second': 5.053, 'train_loss': 0.30488668314615885, 'epoch': 2.0}


TrainOutput(global_step=750, training_loss=0.30488668314615885, metrics={'train_runtime': 148.4379, 'train_samples_per_second': 101.052, 'train_steps_per_second': 5.053, 'train_loss': 0.30488668314615885, 'epoch': 2.0})

El resultado es algo mejor.

## Para pasar a producción

En un caso real guardaríamos el modelo en disco para usarlo después en producción para la inferencia. Lo más habitual es que esto implique **integralo** con los sistemas de información existentes.

Guardamos el modelo en disco:

In [11]:
trainer.save_model("modelo-salud-dental")

Saving model checkpoint to modelo-salud-dental
Configuration saved in modelo-salud-dental/config.json
Model weights saved in modelo-salud-dental/pytorch_model.bin
tokenizer config file saved in modelo-salud-dental/tokenizer_config.json
Special tokens file saved in modelo-salud-dental/special_tokens_map.json


... y lo recuperaríamos en el sistema de información de producción:

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("modelo-salud-dental")

loading configuration file modelo-salud-dental/config.json
Model config BertConfig {
  "_name_or_path": "modelo-salud-dental",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}

loading weights file modelo-salud-dental/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification we

# Inferencia

El resultado de la inferencia (score) es la **probabilidad**, entre cero y uno, de que el texto pertenezca a cada una de las clases.

In [13]:
from transformers import TextClassificationPipeline
inferencia = TextClassificationPipeline(model=model, tokenizer=tokz, return_all_scores=True)

inferencia('Pza 2.3 semierupcionada')

/opt/conda/lib/python3.7/site-packages/transformers/pipelines/text_classification.py:90: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  "`return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality", UserWarning


[{'label': 'LABEL_0', 'score': 0.029643751680850983},
 {'label': 'LABEL_1', 'score': 0.970356285572052}]

Las dos clases están etiquetadas como LABEL_0 y LABEL_1, refiriendose, respectivamente, a si un texto **no pertenece** o **sí pertenence** al campo de la odontología.

# ¿Estamos matando moscas a cañonazos?

Podría pensarse que entrenar un modelo de aprendizaje profundo para resolver esta taréa es **desproporcionado**. ¿No habría formas más sencillas basadas en **programación tradicional**? 

Por ejemplo, podríamos recopilar una **lista de términos odontológicos** y, simplemente, buscar estos términos en los diagnósticos: si un diagnóstico tiene alguno de estos términos, tiene que referirse a la odontología.

¿Dónde podríamos obtener estos términos? Tal vez de un **diccionario de odontología**. Pero es posible que los odontólogos utilicen **abreviaturas y jerga** que no aparezca en un diccionario. Sería mejor obtenerlos de los propios **diagnósticos** de los odontólogos.

Pues bien, si ya tenemos una lista de diagnósticos, muy probablemente sea **más fácil** y **más preciso** entrenar un modelo de aprendizaje máquina. Al fin y al cabo, la parte complicada de esta tecnología es obtener el **conjunto de datos de entrenamiento**.  